In [ ]:
!pip install openai==0.28.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 3.2 MB/s eta 0:00:00


In [ ]:
import openai
from pinecone import Pinecone, ServerlessSpec
import time
import logging
import random
from functools import lru_cache

# Configure logging for detailed error messages
logging.basicConfig(level=logging.DEBUG)

# Pinecone and OpenAI API keys
PINECONE_API_KEY = 'add yours'
PINECONE_ENVIRONMENT = 'add env'
OPENAI_API_KEY = 'add api key'  # Use your OpenAI key here

# Initialize the Pinecone client with the API key
pc = Pinecone(api_key=PINECONE_API_KEY)

# Initialize the index name
index_name = 'rag'

# Check if the index already exists, otherwise create a new one
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric='euclidean',
        spec=ServerlessSpec(
            cloud='aws',
            region=PINECONE_ENVIRONMENT
        )
    )

# Connect to the existing or newly created index
index = pc.Index(index_name)

# Set OpenAI API key
openai.api_key = OPENAI_API_KEY

# Example business documents
docs = [
    {"id": "doc1", "text": "Our business operates from 9 AM to 5 PM."},
    {"id": "doc2", "text": "We offer free shipping for orders over $50."},
]

# Function to convert documents to embeddings using OpenAI's embeddings model with retries and jitter
def get_embeddings_with_retry(text, retries=3):
    for attempt in range(retries):
        try:
            response = openai.Embedding.create(input=[text], model="text-embedding-ada-002")
            return response['data'][0]['embedding']
        except openai.error.RateLimitError:
            wait_time = 2 ** attempt + random.uniform(0, 1)  # Add jitter to avoid rate-limit bottleneck
            logging.warning(f"Rate limit exceeded. Retrying in {wait_time:.2f} seconds...")
            time.sleep(wait_time)
    raise Exception("Failed to get embeddings after several attempts.")

# Cache embeddings for frequently queried documents to reduce API calls
@lru_cache(maxsize=10)
def get_cached_embeddings(doc_id, text):
    return get_embeddings_with_retry(text)

# Insert documents into Pinecone (consider batching for large datasets)
batch_size = 100  # Adjust batch size as needed
for i in range(0, len(docs), batch_size):
    batch = docs[i:i + batch_size]
    embeddings = []
    for doc in batch:
        try:
            embedding = get_cached_embeddings(doc["id"], doc["text"])
            if embedding and len(embedding) == 1536:
                embeddings.append((str(doc["id"]), embedding))
            else:
                raise ValueError(f"Embedding size for {doc['id']} is incorrect or not available.")
        except Exception as e:
            logging.error(f"Error processing document {doc['id']}: {e}")

    # Only attempt to upsert if embeddings are valid and non-empty
    if embeddings:
        try:
            index.upsert(embeddings)
            logging.info(f"Successfully inserted batch {i // batch_size + 1}")
        except Exception as e:
            logging.error(f"Error during upsert: {e}")

print("Documents inserted into Pinecone!")

# Function to retrieve relevant documents from Pinecone
def retrieve_from_pinecone(query):
    # Get query embeddings
    query_embedding = get_embeddings_with_retry(query)

    # Query Pinecone for relevant documents
    results = index.query([query_embedding], top_k=2)

    # Extract and return document IDs
    matches = results['matches']
    docs = [match['id'] for match in matches]

    return docs

# Function to generate an answer using GPT-4 and retrieved documents
def generate_answer(docs, query):
    # Combine documents and query
    context = "\n".join([f"Document: {doc}" for doc in docs])

    # GPT-4 API call for answer generation
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=f"{context}\n\nQuestion: {query}\nAnswer:",
        max_tokens=150
    )

    return response['choices'][0]['text']

# Main RAG-based Q&A bot
def rag_qa_bot(query):
    # Step 1: Retrieve relevant documents from Pinecone
    docs = retrieve_from_pinecone(query)

    # Step 2: Generate an answer using GPT-4
    answer = generate_answer(docs, query)

    return answer

# Example query
query = "What are your business hours?"
answer = rag_qa_bot(query)
print(answer)

ERROR:root:Error processing document doc1: Failed to get embeddings after several attempts.
ERROR:root:Error processing document doc2: Failed to get embeddings after several attempts.


Documents inserted into Pinecone!


Exception: Failed to get embeddings after several attempts.